In [1]:
! pip install -q langchain openai langchain_openai chromadb tiktoken panel
! pip install -q huggingface_hub
! pip install -q sentence_transformers

In [2]:
! pip install -q beautifulsoup4

In [3]:
import os
import re
from bs4 import BeautifulSoup as bs
import requests
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredHTMLLoader, TextLoader, PyPDFLoader, Docx2txtLoader,BSHTMLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter,  CharacterTextSplitter
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import DocArrayInMemorySearch

In [4]:
import pandas as pd
from langsmith import client
from langchain.smith import RunEvalConfig, run_on_dataset

In [5]:
# Load environment variables from .env file
#dotenv_path ="/content/drive/Othercomputers/My laptop/RAG/.env"

dotenv_path = "/.env"

In [6]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(dotenv_path) # read local .env file

In [7]:
# Update with your API URL if using a hosted instance of Langsmith.
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))#"ls__b275bcfbbd564686bb693d4c6e70474a"  # Update with your API key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_TRACING_V2"] ="RAG_QA"

In [8]:
openai_api_key  = os.environ['OPENAI_API_KEY']

In [9]:
#  Access the HUGGINGFACEHUB_API_TOKEN
HUGGINGFACEHUB_API_TOKEN=os.environ["HUGGINGFACEHUB_API_TOKEN"] #huggingfacehub_api_token


In [12]:
# HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [14]:
import os
import requests
from bs4 import BeautifulSoup as bs
import re
import urllib.parse

def scrape_and_save_multiple(urls, folder_path):
    try:
        # Create the folder if it doesn't exist
        os.makedirs(folder_path, exist_ok=True)

        for url in urls:
            # Send a GET request to the URL
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for any HTTP error

            # Parse the HTML content of the page
            soup = bs(response.text, 'html.parser')

            # Extract text content
            text_content = soup.get_text()

            # Remove white spaces
            text_content_no_spaces = re.sub(r'\s+', ' ', text_content)

            # Extract filename from URL
            # Using urllib.parse to handle special characters in URL
            parsed_url = urllib.parse.urlparse(url)
            file_name = os.path.basename(parsed_url.path) + '.txt'

            # Specify the file path
            file_path = os.path.join(folder_path, file_name)

            # Write the text content to a file
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(text_content_no_spaces)

            print(f"Text content from '{url}' saved to '{file_path}'")
    except requests.RequestException as e:
        print(f"Error occurred during HTTP request: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
urls = ['https://www.aru.ac.uk/about-us/find-an-expert?faculty=Science+and+Engineering',
        'https://www.aru.ac.uk/study/subject-areas/computer-science?levelofstudy=Postgraduate',
        'https://www.aru.ac.uk/research/postgraduate-research/degrees-we-offer',
        'https://www.aru.ac.uk/science-and-engineering/research/contact-our-research-team',
        'https://www.aru.ac.uk/science-and-engineering/faculty-staff?location=Cambridge',
        'https://www.aru.ac.uk/about-us/explore-aru/teaching-excellence',
        'https://www.aru.ac.uk/about-us/our-mission-and-values']

folder_path = 'scraped_data'  # Specify the folder path where the text files will be saved

# Call the function
scrape_and_save_multiple(urls, folder_path)


Text content from 'https://www.aru.ac.uk/about-us/find-an-expert?faculty=Science+and+Engineering' saved to 'scraped_data\find-an-expert.txt'
Text content from 'https://www.aru.ac.uk/study/subject-areas/computer-science?levelofstudy=Postgraduate' saved to 'scraped_data\computer-science.txt'
Text content from 'https://www.aru.ac.uk/research/postgraduate-research/degrees-we-offer' saved to 'scraped_data\degrees-we-offer.txt'
Text content from 'https://www.aru.ac.uk/science-and-engineering/research/contact-our-research-team' saved to 'scraped_data\contact-our-research-team.txt'
Text content from 'https://www.aru.ac.uk/science-and-engineering/faculty-staff?location=Cambridge' saved to 'scraped_data\faculty-staff.txt'
Text content from 'https://www.aru.ac.uk/about-us/explore-aru/teaching-excellence' saved to 'scraped_data\teaching-excellence.txt'
Text content from 'https://www.aru.ac.uk/about-us/our-mission-and-values' saved to 'scraped_data\our-mission-and-values.txt'


#Document Loader

In [15]:
path = r"..\RAG_QA_1\scraped_data"
#path = r"C:\Users\User2\Desktop\RAG\scraped_data"
#path = r"C:\Users\Administrator\Desktop\RAG_QA\scraped_data"
loader = DirectoryLoader(path, glob="**/*.txt",loader_cls=TextLoader)
data = loader.load()

In [16]:
# We'll split our data into chunks around 500 characters each with a 50 character overlap. These are relatively small.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1400,
                                               chunk_overlap=300,
                                               separators=["\n\n", "\n", "(?<=\. )", " ", ""])
texts = text_splitter.split_documents(data)

In [17]:
docs = data

In [18]:
single_doc = data[0]
single_doc

Document(page_content=" Computer Science Degrees - Discover ARU Courses - ARU We use cookies to improve your browsing experience, monitor how our site is used, and aid us with advertising. By continuing to use our site you are agreeing to our privacy and cookies policy. Skip main navigation Site search Students and staff Search Menu Study with us Back to main menu Courses Subject areas Undergraduate Postgraduate Postgraduate research College courses Short courses and CPD Degree apprenticeships Distance learning at ARU Apply How to apply - undergraduate How to apply - postgraduate taught How to apply - college courses Admissions process Interviews, auditions and portfolios Discover Open Days Teaching excellence Schools and colleges Community events Writtle College Writtle College Governance Study with us Student life Back to main menu Explore Our campuses Accommodation Open Days Prepare Help with student finances Before you get here Welcome Week Starting your course Information for pare

In [19]:
for doc in docs:
  print(doc.metadata)


{'source': '..\\RAG_QA_1\\scraped_data\\computer-science.txt'}
{'source': '..\\RAG_QA_1\\scraped_data\\contact-our-research-team.txt'}
{'source': '..\\RAG_QA_1\\scraped_data\\degrees-we-offer.txt'}
{'source': '..\\RAG_QA_1\\scraped_data\\faculty-staff.txt'}
{'source': '..\\RAG_QA_1\\scraped_data\\find-an-expert.txt'}
{'source': '..\\RAG_QA_1\\scraped_data\\our-mission-and-values.txt'}
{'source': '..\\RAG_QA_1\\scraped_data\\teaching-excellence.txt'}


In [20]:
#doc.metadata

In [21]:
len(texts)

77

#Embeddings

In [22]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

c:\Users\Administrator\miniconda3\envs\RAG_QA_1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# Persist directory to store the embeddings
persist_directory = "db"

In [24]:
# remove old database files if any
!rm -rf ./db

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [25]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
    )

In [26]:
query = "what is the entry requirement for computer science"
docs = vectordb.similarity_search(query)

In [27]:
from langchain_community.llms import HuggingFaceEndpoint
# from langchain.chains.question_answering import load_qa_chain
# from langchain import HuggingFaceHub


In [28]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [29]:
question = "What is the entry requirement for computer science in ARU ? "

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.

Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature = 0.1,
    model_kwargs={"max_length": 128, "token": HUGGINGFACEHUB_API_TOKEN, "return_source_documents": True}
)
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain.invoke(question)
response = llm_chain.invoke(question)
response

In [ ]:
question = "What is the entry requirement for computer science in ARU ? "
response = llm_chain.invoke(question)
response

{'question': 'What is the entry requirement for computer science in ARU ? ',
 'text': " To study computer science at Anglia Ruskin University (ARU), applicants typically need a minimum of two A-levels or equivalent qualifications. These qualifications should include subjects like Mathematics and Physics or another science subject. However, the exact requirements may vary depending on the specific course and individual circumstances, so it's best to check the ARU website or contact their admissions office for the most accurate information. Thanks for asking!"}

In [ ]:
question = "What degrees are offered by ARU at postgraduate level?"
response = llm_chain.invoke(question)
response['text']

" Anglia Ruskin University (ARU) provides a wide range of postgraduate programs. These include Master's degrees, Master of Research (MRes), Master of Philosophy (MPhil), and Doctoral degrees. Thanks for asking!"

In [ ]:
from langchain.chains import QAGenerationChain

In [ ]:
chain = QAGenerationChain.from_llm(ChatOpenAI(temperature=0))
qa = chain.run(data[0].page_content)
qa_2 = chain.run(data[1].page_content)

c:\Users\Administrator\miniconda3\envs\RAG_QA_1\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
examples = [qa[0], qa_2[0]]
examples

[{'question': 'What is the purpose of using cookies on the ARU website?',
  'answer': 'To improve browsing experience, monitor site usage, and aid with advertising.'},
 {'question': 'What is one of the purposes mentioned for using cookies on the website?',
  'answer': 'To improve browsing experience, monitor site usage, and aid with advertising.'}]

In [ ]:
chain = QAGenerationChain.from_llm(ChatOpenAI(temperature=0))
qa_3 = chain.invoke(data[0].page_content)
qa_4 = chain.invoke(data[1].page_content)

In [ ]:
print(qa_3.keys())
print(qa_4.keys())

dict_keys(['text', 'questions'])
dict_keys(['text', 'questions'])


In [ ]:
examples3 = [qa_3['questions']]
example4 = [qa_4['questions']]
examples3


[[{'question': 'What is the purpose of using cookies on the ARU website?',
   'answer': 'To improve browsing experience, monitor site usage, and aid with advertising.'},
  {'question': 'What types of degrees are offered for postgraduate research at ARU?',
   'answer': 'Postgraduate research degrees'},
  {'question': 'What skills can students gain by studying the Cyber Security Postgraduate - MSc course in Cambridge?',
   'answer': 'Students can gain skills in programming, networking, hardware, and analysis by studying the Cyber Security Postgraduate - MSc course in Cambridge.'},
  {'question': 'What A levels do you need for Computer Science?',
   'answer': 'For the majority of Computer Science courses, you need a certain number of UCAS tariff points but it does not matter which A levels (or equivalent qualifications) they come from. STEM-related subjects would be a benefit, including Mathematics and Computing or Physics.'},
  {'question': 'What is the average starting salary for Comput

In [ ]:
example4

[[{'question': 'What is one of the purposes mentioned for using cookies on the ARU website?',
   'answer': 'To improve browsing experience, monitor site usage, and aid with advertising.'},
  {'question': 'What types of degrees are offered for postgraduate research at ARU?',
   'answer': 'Professional Doctorate in Science and Engineering and PhD project opportunities'},
  {'question': 'Where is the Research Administration Manager located?',
   'answer': 'Cambridge campus, Science Centre, room SCI708'}]]

#Evaluation

In [32]:
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="ls__b275bcfbbd564686bb693d4c6e70474a"
LANGCHAIN_PROJECT="RAG_QA"

In [33]:
client = client.Client()

In [ ]:
# from langchain.chat_models import ChatOpenAI

# p_llm = ChatOpenAI()
# p_llm.invoke("Hello, world!")

In [ ]:
# inputs= ['What is the purpose of using cookies on the ARU website?',
#         'What types of skills can students learn in the Artificial Intelligence Postgraduate - MSc course at ARU?',
#         'What qualifications are required to study Computer Science at ARU?',
#         'What A levels do I need for Computer Science?',
#         'What are some jobs that one can get with a Computer Science degree?']

# outputs = ['To improve browsing experience, monitor site usage, and aid with advertising.',
#          'Students can learn in-demand skills in data analysis and modelling in the Artificial Intelligence Postgraduate - MSc course at ARU.',
#          'To study for a Computer Science BSc (Hons) at ARU, you will need 104 UCAS tariff points from a minimum of 2 A Levels and 3 GCSEs at grade C, or grade 4, or above, including English and Mathematics. For a Computer Science BEng (Hons), you will need 112 UCAS tariff points from a minimum of 2 A Levels and 3 GCSEs at grade C, or grade 4, or above, including English and Mathematics. Equivalent qualifications such as BTEC Level 3 Diplomas, International Baccalaureate Diploma, Access to HE Diplomas, and others are also accepted.',
#          'For the majority of Computer Science courses, you need a certain number of UCAS tariff points but it does not matter which A levels (or equivalent qualifications) they come from. STEM-related subjects would be a benefit, including Mathematics and Computing or Physics.',
#          'Some jobs that one can get with a Computer Science degree include Application analyst, Applications developer, Cyber security analyst, Data analyst, Forensic computer analyst, Game designer, Games developer, Machine learning engineer, Penetration tester, Software engineer, Systems analyst, UX designer, Web designer, and Web developer.',
#          ]

# # Dataset

# qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
# df = pd.DataFrame(qa_pairs)


In [ ]:
# inputs= ['What skills can I learn in the Artificial Intelligence Postgraduate program at ARU?',
#         'Are there opportunities for practical experience in the Computer Science Postgraduate program at ARU?',
#         'What is the duration of the Cyber Security Postgraduate program at ARU?',
#         'What subjects are covered in the Information and Communication Technology (Conversion) Postgraduate program at ARU?',
#         'What are some examples of jobs directly related to a Computer Science degree from ARU?',
#         'Who is the Director of Research Students at ARU?',
#         'Who is the Senior Research Administrator located at the Chelmsford campus?',
#         'What types of research degrees are offered at ARU?',
#         'What is the typical duration for full-time PhD candidates at ARU?',
#         'How can someone apply for a PhD by Published Work at ARU?',
#         'How many members of staff are available in the Faculty of Science and Engineering at ARU"s Cambridge campus?',
#         'Who specializes in Cancer biology among the staff at ARU"s Cambridge campus?']

# outputs = ['In the Artificial Intelligence Postgraduate program, you can learn in-demand skills in data analysis and modeling.',
#          'Yes, knowledge from the teaching has practical applications, also ARU support student with taking up work placement to gain practical industry experience.',
#          'The Cyber Security Postgraduate program is available as a 12-month full-time course or a 28-month part-time course.',
#          'The Information and Communication Technology (Conversion) program covers programming, networking, hardware, and analysis.',
#          'Jobs directly related to a Computer Science degree include software engineer, data analyst, web developer, and cybersecurity analyst.',
#          'The Director of Research Students at ARU is Dr. Nezhapi-Dellé Odeleye.',
#          'The Senior Research Administrator located at the Chelmsford campus is Sam Wilson.',
#          'ARU offers Doctor or Master of Philosophy (PhD/MPhil), MD (Res), PhD by Published Work, and Professional Doctorate programs.',
#          'Full-time PhD candidates at ARU typically complete their program within three to four years.',
#          'To apply for a PhD by Published Work at ARU, individuals must demonstrate an independent and original contribution to knowledge through published work within the last ten years.',
#          'There are 244 members of staff available at ARU"s Cambridge campus in the Faculty of Science and Engineering.',
#          'Hussein Al-Ali is a Lecturer in the Faculty of Science and Engineering, working in the School of Life Sciences at ARU"s Cambridge campus, specializing in Cancer biology.'
#          ]

# # Dataset

# qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
# df = pd.DataFrame(qa_pairs)


In [ ]:
# inputs= ['Which bodies accredit the computer science degrees at ARU?',
#          'Which college did ARU recently merged with?',
#          'What skill will PhD candidates be able to demonstrate?',
#          'What are the professional doctorates currently on offer at ARU? ',
#          'Who is the Deputy Dean for Research and Innovation?',
#          'What is the vision of ARU?',
#          'What is the first part of the professional doctorate?',
#          'How is ARU rated by the  Teaching Excellence Framework (TEF)',
#          'What ARU Employability Strategy help students do?',
#          'What percentage of ARU staff were recognised  by HEA in 2017-2018',
#          'Do you need maths to study artificial intelligence at ARU?',
#          'What is the best course for Computer Science?']

# outputs = ['BCS and the IET.',
#            'Writtle University College.',
#            'PhD candidate, you"ll also be able to demonstrate an understanding of research methods appropriate to your field of research.',
#            'Doctor of Business Administration (DBA), Education (EdD), Health and Social Care, Practical Theology, Science and Engineering.',
#            'Dr Nick Pugh',
#            'Transforming lives through innovative, inclusive and entrepreneurial education and research',
#            'The first part of the Professional Doctorate is a taught programme.',
#            ' Gold award ',
#            'helps students to build the skills and abilities that employers value in graduates.',
#            'In 2017-2018 the percentage of our teaching staff who were recognised by the HEA was over 78%',
#            'To study Artificial Intelligence you will need a pass in Mathematics at grade E or above',
#            'The best course for Computer Science is the one that covers all the basics of how computers work, how they communicate, how they can be programmed and various other topics around data storage, data structures, and general software security.']

# # Dataset

# qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
# df = pd.DataFrame(qa_pairs)


#LLM QA

In [ ]:
# inputs = ["What is the purpose of using cookies on the ARU website?",
#          "What types of degrees are offered for postgraduate research at ARU?",
#          "What skills can students gain by studying the Information and Communication Technology (Conversion) Postgraduate - MSc course in Cambridge?",
#          "What A levels do you need for Computer Science at this university?",
#          "What is the average starting salary for Computer Science graduates in the UK?",
#          "What are some of the research institutes mentioned in the text?",
#          "What is the purpose of using cookies on the ARU website?",
#          "According to the text, what are the values of Writtle University College and ARU?",
#          "What considerations does the text mention that guide decision-making and actions?",
#          "What types of research degrees are offered at ARU?",
#          "What types of degrees can you undertake at ARU for postgraduate research?",
#          "What are the benefits of studying a Professional Doctorate?",
#          "What are the two stages involved in the Professional Doctorate program?",
#          "What is the purpose of using cookies on the Faculty of Science and Engineering staff website?",
#          "What types of degrees does ARU offer for postgraduate research?",
#          "What is Hussein Al-Ali's expertise?",
#          "What is one of the purposes mentioned for the use of cookies on the website?",
#          "What types of degrees are offered for postgraduate research at ARU?",
#          "Where is the Research Administration Manager located?",
#          "What is one of the ways in which ARU aims to enhance the student experience?",
#          "What is the Teaching Excellence Framework (TEF) and what does a Gold award signify?",
#          "What is the Teaching Excellence Framework (TEF) and what are its outcomes?",
#          "What is the purpose of the Welcome Buddies scheme mentioned in the text?",
#          "What types of courses are offered at ARU for international students?",
#          "What expertise does Hossein Abroshan have?",
#          "What is the expertise of Imran Ahmed, a Senior Lecturer at the Faculty of Science and Engineering?"]



# outputs = ["To improve browsing experience, monitor site usage, and aid with advertising.",
#           "Postgraduate research degrees",
#           "Students can gain skills in programming, networking, hardware, and analysis by studying the Information and Communication Technology (Conversion) Postgraduate - MSc course in Cambridge.",
#           "You will need a minimum of 104 UCAS tariff points from 2 A Levels and 3 GCSEs at grade C or above, including English and Mathematics.",
#           "£25,000",
#           "Cambridge Institute for Music Therapy Research, Global Sustainability Institute, Policing Institute for the Eastern Region, StoryLab, Veterans & Families Institute for Military Social Research, Vision and Eye Research Institute",
#           "To improve browsing experience, monitor site usage, and aid with advertising.",
#           "The values of Writtle University College and ARU are Ambition, Innovation, Courage, Community, Integrity, and Responsibility.",
#           "The text mentions that decisions and actions are guided by environmental, societal, and economic considerations.",
#           "Postgraduate research degrees",
#           "At ARU, you can undertake a Doctor or Master of Philosophy (PhD/MPhil), an MD (Res), a PhD by Published Work, or a Professional Doctorate.",
#           "By studying a Professional Doctorate, you will research an area of particular interest to you and your employer, gain an internationally recognised doctorate qualification, make a significant contribution through your research findings to your practice, enhance your profile, and expand and strengthen your professional network.",
#           "Stage 1 involves workshops and seminars on methodology and research design, leading to the preparation of papers totaling a maximum of 21,000 words. Stage 2 involves undertaking original research resulting in a thesis.",
#           "To improve browsing experience, monitor site usage, and aid with advertising.",
#           "ARU offers postgraduate research degrees in Health, Performance and Wellbeing, Safe and Inclusive Communities, and Sustainable Futures.",
#           "Cancer biology",
#           "To improve browsing experience, monitor site usage, and aid with advertising.",
#           "Professional Doctorate in Science and Engineering and PhD project opportunities",
#           "Cambridge campus, Science Centre, room SCI708",
#           "Teaching excellence",
#           "The Teaching Excellence Framework (TEF) is a national scheme that focuses on teaching, learning, and achieving positive outcomes from studies. A Gold award signifies outstanding course content and delivery, innovative and highly effective course design and content, and other areas of excellence.",
#           "The Teaching Excellence Framework (TEF) is a national scheme run by the Office for Students that focuses on teaching, learning, and achieving positive outcomes from studies. Its outcomes are Bronze, Silver, and Gold, with a Gold award signifying that the 'student experience and outcomes are typically outstanding.'",
#           "The purpose of the Welcome Buddies scheme is to pair new undergraduate students with a Buddy to help them settle into university life.",
#           "Courses for international students, Study Abroad and Exchange, International student support, English language support",
#           "Cybersecurity and Networking, Artificial Intelligence",
#           "Artificial Intelligence, Computer Science"]


# # Dataset

# qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
# df = pd.DataFrame(qa_pairs)

#Manual QA

In [34]:
inputs = ["What types of research degrees are offered at ARU?",
         "What is the duration of the Cyber Security Postgraduate program at ARU?",
         "What are some examples of jobs directly related to a Computer Science degree from ARU?",
         "What A levels do I need for Computer Science?",
         "What types of skills can students learn in the Artificial Intelligence Postgraduate - MSc course at ARU?",
         "Do you need maths to study artificial intelligence at ARU?",
         "Which bodies accredit the computer science degrees at ARU?",
         "What skill will PhD candidates be able to demonstrate?",
         "What is the vision of ARU?",
         "What does ARU Employability Strategy help students do?",]

outputs = ["ARU offers Doctor or Master of Philosophy (PhD/MPhil), MD (Res), PhD by Published Work, and Professional Doctorate programs.",
          "The Cyber Security Postgraduate program is available as a 12-month full-time course or a 28-month part-time course.",
          "Jobs directly related to a Computer Science degree include software engineer, data analyst, web developer, and cybersecurity analyst.",
          "For the majority of Computer Science courses, you need a certain number of UCAS tariff points but it does not matter which A levels (or equivalent qualifications) they come from. STEM-related subjects would be a benefit, including Mathematics and Computing or Physics.",
          "Students can learn in-demand skills in data analysis and modelling in the Artificial Intelligence Postgraduate - MSc course at ARU.",
          "To study Artificial Intelligence you will need a pass in Mathematics at grade E or above",
          "BCS and the IET.",
          "PhD candidate, you'll also be able to demonstrate an understanding of research methods appropriate to your field of research.",
          "Transforming lives through innovative, inclusive and entrepreneurial education and research.",
          "helps students to build the skills and abilities that employers value in graduates."]


qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
df = pd.DataFrame(qa_pairs)

In [35]:
# Write to csv
csv_path = "Manual_QA.csv"
df.to_csv(csv_path, index=False)

In [36]:
from langsmith import Client

client = Client()
dataset_name = "Manual_QA"

# Store
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Manual QA pairs.",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

#Update dataset

In [37]:
# OpenAI API

import openai
from langsmith.wrappers import wrap_openai
openai_client = wrap_openai(openai.Client())

def answer_Website_QA_question_oai(inputs: dict) -> dict:
    """
    Generates answers to user questions based on a provided website text using OpenAI API.

    Parameters:
    inputs (dict): A dictionary with a single key 'question', representing the user's question as a string.

    Returns:
    dict: A dictionary with a single key 'output', containing the generated answer as a string.
    """

    # System prompt
    system_msg = f"Answer user questions in 2-3 sentences about this context: \n\n\n {docs}"

    # Pass in website text
    messages = [{"role": "system", "content": system_msg},
                {"role": "user", "content": inputs["question"]}]

    # Call OpenAI
    response = openai_client.chat.completions.create(messages=messages, model="gpt-3.5-turbo")

    # Response in output dict
    return {"answer": response.dict()['choices'][0]['message']['content']}

In [43]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# Evaluators
qa_evalulator = [LangChainStringEvaluator("qa")]
dataset_name = "Manual_QA"

experiment_results = evaluate(
    answer_Website_QA_question_oai,
    data=dataset_name,
    evaluators=qa_evalulator,
    experiment_prefix="AUTO_LLM_Generated-qa-openai",
    # Any experiment metadata can be specified here
    metadata={
      # "variant": "stuff website context into gpt-3.5-turbo",
    },
)

View the evaluation results for experiment: 'AUTO_LLM_Generated-qa-openai-a727e7d5' at:
https://smith.langchain.com/o/1a0b86e1-40c4-5832-b964-0beecf9d2cbd/datasets/349b1855-bcf4-43e9-8cc4-d237a256ccf8/compare?selectedSessions=be85ddd2-c57d-44c9-b7cd-0a353a585a5d




10it [00:07,  1.27it/s]


In [39]:
! pip install -U langsmith

In [40]:
!pip install -q ollama

In [41]:
# Mistral

import ollama
from langsmith.run_helpers import traceable

@traceable(run_type="llm")
def call_ollama(messages, model: str):
    stream = ollama.chat(messages=messages, model='mistral', stream=True)
    response = ''
    for chunk in stream:
        print(chunk['message']['content'], end='', flush=True)
        response =  response + chunk['message']['content']
    return response

def answer_website_question_mistral(inputs: dict) -> dict:
    """
    Generates answers to user questions based on a provided website text using Ollama serving Mistral locally.

    Parameters:
    inputs (dict): A dictionary with a single key 'question', representing the user's question as a string.

    Returns:
    dict: A dictionary with a single key 'output', containing the generated answer as a string.
    """

    # System prompt
    system_msg = f"Answer user questions about this context: \n\n\n {docs}"

    # Pass in website text
    messages = [{"role": "system", "content": system_msg},
                {"role": "user", "content": f'Answer the question in 2-3 sentences {inputs["question"]}' }]

    # Call Mistral
    response = call_ollama(messages, model="mistral")

    # Response in output dict
    return {"answer": response}



In [44]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
# Evaluators
qa_evalulator = [LangChainStringEvaluator("qa")]
dataset_name = "Manual_QA"

experiment_results = evaluate(
    answer_website_question_mistral,
    data=dataset_name,
    evaluators=qa_evalulator,
    experiment_prefix="Manual-qa-mistral",
    # Any experiment metadata can be specified here
    metadata={
      "variant": "website context into mistral",
    },)

View the evaluation results for experiment: 'Manual-qa-mistral-f4325062' at:
https://smith.langchain.com/o/1a0b86e1-40c4-5832-b964-0beecf9d2cbd/datasets/349b1855-bcf4-43e9-8cc4-d237a256ccf8/compare?selectedSessions=4808dfaa-374e-4e34-976b-b0743cd068ba




0it [00:00, ?it/s]

 Anglia Ruskin University (ARU) offers both Bachelor's and Master's degrees in Computer Science, specifically a BSc (Hons) and a BEng (Hons) in Computer Science. These degrees do not require any specific prior knowledge or qualifications beyond A-Levels, but Mathematics and science-related subjects are recommended for advanced study. The entry requirements include certain UCAS tariff points from A-Levels and GCSEs, along with English language proficiency if applicable. The document does not provide information about the duration of the Cyber Security Postgraduate program at ARU. However, it mentions that ARU offers Computer Science courses, including a BSc (Hons) and a BEng (Hons), which require 3 years of full-time study. It's recommended to check the specific Cyber Security postgraduate course webpage or contact ARU for accurate information on its length. A Computer Science degree from Anglia Ruskin University (ARU) can lead to various jobs in the tech industry, including software en

10it [09:59, 59.91s/it]
